# 层和块

我们先回顾一下多层感知机

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1445,  0.0104, -0.0149, -0.1505,  0.2127,  0.1162,  0.2087,  0.2139,
         -0.0640,  0.1559],
        [-0.1402,  0.0562,  0.0159, -0.3024,  0.1329,  0.0990,  0.1825, -0.0074,
         -0.1688,  0.0150]], grad_fn=<AddmmBackward0>)

在这个例子中，我们通过实例化nn.Sequential来构建我们的模型， 层的执行顺序是作为参数传递的。 简而言之，nn.Sequential定义了一种特殊的Module， 即在PyTorch中表示一个块的类， 它维护了一个由Module组成的有序列表。 注意，两个全连接层都是Linear类的实例， Linear类本身就是Module的子类。 另外，到目前为止，我们一直在通过net(X)调用我们的模型来获得模型的输出。 这实际上是net.__call__(X)的简写。 这个前向传播函数非常简单： 它将列表中的每个块连接在一起，将每个块的输出作为下一个块的输入。

`nn.Sequential`定义了一种特殊的`Module`

### 自定义块:
将输入数据作为其前向传播函数的参数。

通过前向传播函数来生成输出。请注意，输出的形状可能与输入的形状不同。例如，我们上面模型中的第一个全连接的层接收一个20维的输入，但是返回一个维度为256的输出。

计算其输出关于输入的梯度，可通过其反向传播函数进行访问。通常这是自动发生的。

存储和访问前向传播计算所需的参数。

根据需要初始化模型参数。

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

torch.nn.ReLU 是一个模块类，通常用于构建序列模型（即 nn.Sequential 模块）或者在定义自己的 nn.Module 子类时将其作为一个层包含在内。

torch.nn.functional.relu 是一个函数，通常在定义自己的网络模块时，在 forward 方法中直接调用来应用激活函数。

总结一下，nn.ReLU是一个层，需要在构建模型的时候就定义好，而F.relu是一个函数，只能在前向传播的函数中被调用，它更加灵活，因为它不需要事先定义模型。这两者都是内置ReLU激活函数的选择，而使用哪一种取决于个人的代码风格和特定的用例。

实例化多层感知机的层，然后在每次调用前向传播函数时调用这些层

In [4]:
net = MLP()
net(X)

tensor([[ 0.1635, -0.2636, -0.0760, -0.2374,  0.1898,  0.0729, -0.0105,  0.1855,
          0.0559, -0.0855],
        [ 0.1519, -0.1852,  0.0123, -0.1386,  0.0088,  0.0129,  0.0009,  0.1650,
         -0.0139, -0.0306]], grad_fn=<AddmmBackward0>)

### 顺序块
现在我们可以更仔细地看看Sequential类是如何工作的， 回想一下Sequential的设计是为了把其他模块串起来。 为了构建我们自己的简化的MySequential， 我们只需要定义两个关键函数：

一种将块逐个追加到列表中的函数；

一种前向传播函数，用于将输入按追加块的顺序传递给块组成的“链条”。

下面的MySequential类提供了与默认Sequential类相同的功能。

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            # str(idx)通过字典的键索引
            # _名字变量是内部使用的变量
            self._modules[str(idx)] = module

    def forward(self, X):
        # .values()获得字典的values
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1425,  0.1552, -0.0698,  0.0215,  0.4182,  0.4846, -0.0090, -0.2805,
          0.0133, -0.1630],
        [ 0.1597,  0.0818, -0.1490,  0.1249,  0.3177,  0.3237,  0.1127, -0.2987,
          0.0641,  0.0255]], grad_fn=<AddmmBackward0>)

在前向传播函数中执行代码

 然而，有时我们可能希望合并既不是上一层的结果也不是可更新参数的项， 我们称之为常数参数（constant parameter）。 例如，我们需要一个计算函数f(x, w)=c*wTx 的层，其中x是输入,w是参数，T进行转置,c是某个在优化过程中没有更新的指定常量。 因此我们实现了一个FixedHiddenMLP类，如下所示：

In [6]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # torch.mm()对两个矩阵进行矩阵乘法操作
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(0.3631, grad_fn=<SumBackward0>)

在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在L1
范数大于1
的条件下， 将输出向量除以2
，直到它满足条件为止。 最后，模型返回了X中所有项的和。 注意，此操作可能不会常用于在任何实际任务中， 我们只展示如何将任意代码集成到神经网络计算的流程中。

### 混合搭配各种组合块的方法

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0850, grad_fn=<SumBackward0>)